In [1]:
import open3d as o3d
import numpy as np
import os
from open3d.web_visualizer import draw

from utils.depth_utils import *
from utils.register_utils import *

output_dir = "output"

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] Resetting default logger to print to terminal.


In [5]:
# Register the mesh to the original point cloud
print("[INFO] Registering mesh to original point cloud...")

# object_id = args.obj_id
object_id = "06830"\

# np.random.seed(4)

# Load the ground truth mesh
complete_pcd = o3d.io.read_triangle_mesh(f"./redwood_dataset/GT/{object_id}.ply")
complete_pcd = complete_pcd.sample_points_uniformly(number_of_points=16384)
complete_pcd.points = o3d.utility.Vector3dVector(-np.asarray(complete_pcd.points))
translate = -complete_pcd.get_center()
complete_pcd = complete_pcd.translate(translate)
scale = 0.5 / np.max(np.linalg.norm(np.asarray(complete_pcd.points), axis=1))
complete_pcd = complete_pcd.scale(scale, center=complete_pcd.get_center())

# Load the partial point cloud
partial_pcd = o3d.io.read_point_cloud(f"./redwood_dataset/point_clouds/{object_id}.ply")
partial_pcd.points = o3d.utility.Vector3dVector(-np.asarray(partial_pcd.points))
partial_pcd = partial_pcd.translate(translate)
partial_pcd = partial_pcd.scale(scale, center=complete_pcd.get_center())
partial_pcd = partial_pcd.farthest_point_down_sample(num_samples=16384)

# Load the generated mesh
mesh = o3d.io.read_triangle_mesh(os.path.join(output_dir, f"{object_id}_mesh.ply"))
# Normalize the mesh
mesh = mesh.translate(-mesh.get_center())
mesh = mesh.scale(0.5 / np.max(np.linalg.norm(np.asarray(mesh.vertices), axis=1)), center=mesh.get_center())
mesh_pcd = mesh.sample_points_uniformly(number_of_points=16384)
mesh_pcd.points = o3d.utility.Vector3dVector(np.asarray(mesh_pcd.points) * np.array([-1, 1, 1]))

# Visualize the partial point cloud and the mesh
mesh_pcd.paint_uniform_color([0,0,1])
partial_pcd.paint_uniform_color([1, 0, 0])
complete_pcd.paint_uniform_color([0, 1, 0])

# Draw the partial point cloud and the mesh
draw([partial_pcd, mesh_pcd, complete_pcd])

[INFO] Registering mesh to original point cloud...
[Open3D INFO] Window window_5 created.


WebVisualizer(window_uid='window_5')

In [3]:
from utils.gd_register_utils import gd_registration
mesh_pcd = gd_registration(mesh_pcd, partial_pcd, fps_sample=2048, device='cuda')

# mesh_pcd.transform(transformation)

# Compute the chamfer distance
cd_o3d = (np.mean(complete_pcd.compute_point_cloud_distance(mesh_pcd)) + \
    np.mean(mesh_pcd.compute_point_cloud_distance(complete_pcd))) / 2
print(f"Chamfer Distance: {cd_o3d}")

draw([partial_pcd, mesh_pcd, complete_pcd])
draw([partial_pcd, mesh_pcd])

Target point cloud shape: torch.Size([2048, 3])
Source point cloud shape: torch.Size([2048, 3])
Stage 1: Joint optimization of rotation, translation, and scale
Iteration    0: Loss = 0.002700
Iteration  100: Loss = 0.000844
Iteration  200: Loss = 0.000888
Iteration  300: Loss = 0.000853
Iteration  400: Loss = 0.000845
Iteration  500: Loss = 0.000858
Iteration  600: Loss = 0.000846
Iteration  700: Loss = 0.000856
Iteration  800: Loss = 0.000676
Iteration  900: Loss = 0.000674
Stage 2: Only optimize translation and scale
Iteration    0: Loss = 0.000674
Iteration  100: Loss = 0.000674
Iteration  200: Loss = 0.000674
Optimization complete.
Learned rotation angles (radians): [-2.2643516   1.2678295   0.31350458]
Learned translation: [ 0.09549507 -0.01365601 -0.10897619]
Learned scale: [1.1386921]
Chamfer Distance: 0.09916690991421562
[Open3D INFO] Window window_1 created.


WebVisualizer(window_uid='window_1')

[Open3D INFO] Window window_2 created.


WebVisualizer(window_uid='window_2')

In [8]:
import numpy as np
import torch
import torch.optim as optim
import fpsample
from utils.chamfer_python import distChamfer

def euler_angles_to_rotation_matrix(theta):
    """
    Convert a tensor of 3 Euler angles (in radians) to a 3x3 rotation matrix.
    theta = [alpha, beta, gamma] corresponds to rotations about x, y, and z axes.
    """
    one = torch.tensor(1., device=theta.device, dtype=theta.dtype)
    zero = torch.tensor(0., device=theta.device, dtype=theta.dtype)
    
    cos_a = torch.cos(theta[0])
    sin_a = torch.sin(theta[0])
    cos_b = torch.cos(theta[1])
    sin_b = torch.sin(theta[1])
    cos_c = torch.cos(theta[2])
    sin_c = torch.sin(theta[2])
    
    # Rotation about x-axis
    R_x = torch.stack([
        torch.stack([one,      zero,   zero]),
        torch.stack([zero, cos_a, -sin_a]),
        torch.stack([zero, sin_a,  cos_a])
    ])
    
    # Rotation about y-axis
    R_y = torch.stack([
        torch.stack([cos_b, zero, sin_b]),
        torch.stack([zero,  one,  zero]),
        torch.stack([-sin_b, zero, cos_b])
    ])
    
    # Rotation about z-axis
    R_z = torch.stack([
        torch.stack([cos_c, -sin_c, zero]),
        torch.stack([sin_c,  cos_c, zero]),
        torch.stack([zero,   zero,  one])
    ])
    
    # Combined rotation matrix: using Z-Y-X order
    R = torch.mm(R_z, torch.mm(R_y, R_x))
    return R

def transform_points(points, angles, translation, scale):
    """
    Apply a rigid transformation with scale to the points.
    points: Tensor of shape (N, 3)
    angles: Tensor of shape (3,)
    translation: Tensor of shape (3,)
    scale: Tensor of shape (1,) representing uniform scale.
    Returns the transformed point cloud.
    """
    R = euler_angles_to_rotation_matrix(angles)
    zero = torch.tensor(0., device=points.device, dtype=points.dtype)
    one = torch.tensor(1., device=points.device, dtype=points.dtype)
    # Using matrix multiplication: note that we use the transpose of R so that each point is rotated properly.
    transformation = torch.stack([torch.stack([R[0, 0], R[0, 1], R[0, 2], translation[0]]),
                                  torch.stack([R[1, 0], R[1, 1], R[1, 2], translation[1]]),
                                  torch.stack([R[2, 0], R[2, 1], R[2, 2], translation[2]]),
                                  torch.stack([zero, zero, zero, one])])
    points_homogeneous = torch.cat([points, torch.ones(points.shape[0], 1, device=points.device)], dim=1)  # shape: (N, 4)
    transformed = torch.matmul(points_homogeneous, transformation.T)[:, :3]  # shape: (N, 3)
    transformed = transformed * scale  # Apply uniform scaling
    return transformed

def register_points_gd(source_pc, target_pc, num_iterations=500, learning_rate=0.5):
    """
    Register the source point cloud to the target point cloud using gradient descent.
    
    Parameters:
        source_pc (torch.Tensor): Source point cloud (partial) of shape (N, 3).
        target_pc (torch.Tensor): Target point cloud (complete) of shape (M, 3).
        num_iterations (int): Number of optimization iterations.
        learning_rate (float): Learning rate for the optimizer.
    
    Returns:
        angles, translation, scale: Learned parameters.
    """
    # Initialize the learnable parameters
    angles = torch.nn.Parameter(torch.zeros(3, device=device))
    translation = torch.nn.Parameter(torch.zeros(3, device=device))
    scale = torch.nn.Parameter(torch.ones(1, device=device))
    
    # Best results
    best_loss = float('inf')
    best_angles = angles.clone()
    best_translation = translation.clone()
    best_scale = scale.clone()

    # Set up the optimizer
    # optimizer = optim.SGD([angles, translation, scale], lr=learning_rate, momentum=0.0)
    optimizer = optim.Adam([angles, translation, scale], lr=learning_rate)

    for iteration in range(num_iterations):
        optimizer.zero_grad()
        
        # Apply the transformation to the source point cloud
        transformed_source = transform_points(source_pc, angles, translation, scale)
        
        # Compute the Chamfer loss between the transformed source and target point clouds
        cd_1, cd_2, _, _ = distChamfer(transformed_source.unsqueeze(0), target_pc.unsqueeze(0))
        loss = cd_2.mean()  # Mean Chamfer distance
        
        # Backpropagation
        loss.backward()
        # Multiply the gradient of the scales by 0.1 to make it less sensitive
        scale.grad *= 0.1
             
        optimizer.step()
        
        if iteration % 100 == 0:
            print(f"Iteration {iteration:4d}: Loss = {loss.item():.6f}")
        
        if loss.item() < best_loss:
            best_loss = loss.item()
            best_angles = angles.clone()
            best_translation = translation.clone()
            best_scale = scale.clone()

    return best_angles, best_translation, best_scale

# Assume you have two point clouds:
# `source_pc` is the partial point cloud that you want to register to `target_pc` (the complete point cloud).
# They should be torch tensors of shape (N, 3) and (M, 3) respectively.
#
# For demonstration, here we create synthetic point clouds.
# In practice, replace these with your real data.
torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

target_pc_idx = fpsample.fps_sampling(np.asarray(partial_pcd.points), 2048)
source_pc_idx = fpsample.fps_sampling(np.asarray(mesh_pcd.points), 2048)
target_pc = torch.tensor(np.asarray(partial_pcd.points)[target_pc_idx], dtype=torch.float32, device=device)
source_pc = torch.tensor(np.asarray(mesh_pcd.points)[source_pc_idx], dtype=torch.float32, device=device)
print(f"Target point cloud shape: {target_pc.shape}")
print(f"Source point cloud shape: {source_pc.shape}")

# Register the point clouds
angles, translation, scale = register_points_gd(source_pc, target_pc, num_iterations=1000, learning_rate=0.1)

print("Optimization complete.")
print("Learned rotation angles (radians):", angles.data.cpu().numpy())
print("Learned translation:", translation.data.cpu().numpy())
print("Learned scale:", scale.data.cpu().numpy())

# Plot the results
R = euler_angles_to_rotation_matrix(angles)
zero = torch.tensor(0., device=angles.device, dtype=angles.dtype)
one = torch.tensor(1., device=angles.device, dtype=angles.dtype)
# Using matrix multiplication: note that we use the transpose of R so that each point is rotated properly.
transformation = torch.stack([torch.stack([R[0, 0], R[0, 1], R[0, 2], translation[0]]),
                                torch.stack([R[1, 0], R[1, 1], R[1, 2], translation[1]]),
                                torch.stack([R[2, 0], R[2, 1], R[2, 2], translation[2]]),
                                torch.stack([zero, zero, zero, one])]).detach().cpu().numpy()
scale = scale.detach().cpu().numpy()

# Visualize the partial point cloud and the mesh
mesh_pcd_registered = copy.deepcopy(mesh_pcd)
mesh_pcd_registered = mesh_pcd_registered.transform(transformation)
mesh_pcd_registered.points = o3d.utility.Vector3dVector(np.asarray(mesh_pcd_registered.points) * scale)

# Compute the chamfer distance
cd_o3d = (np.mean(complete_pcd.compute_point_cloud_distance(mesh_pcd_registered)) + \
    np.mean(mesh_pcd_registered.compute_point_cloud_distance(complete_pcd))) / 2
print(f"Chamfer Distance: {cd_o3d}")

draw([partial_pcd, mesh_pcd_registered, complete_pcd])
draw([partial_pcd, mesh_pcd_registered])

Target point cloud shape: torch.Size([2048, 3])
Source point cloud shape: torch.Size([2048, 3])
Iteration    0: Loss = 0.002779
Iteration  100: Loss = 0.000745
Iteration  200: Loss = 0.000745
Iteration  300: Loss = 0.000746
Iteration  400: Loss = 0.000753
Iteration  500: Loss = 0.000759
Iteration  600: Loss = 0.000749
Iteration  700: Loss = 0.000753
Iteration  800: Loss = 0.000750
Iteration  900: Loss = 0.000775
Optimization complete.
Learned rotation angles (radians): [-0.93191963  0.0627753  -0.6163153 ]
Learned translation: [ 0.04295043 -0.03262854 -0.05272727]
Learned scale: [0.9583571]
Chamfer Distance: 0.04756831151413324
[Open3D INFO] Window window_10 created.


WebVisualizer(window_uid='window_10')

[Open3D INFO] Window window_11 created.


WebVisualizer(window_uid='window_11')

In [18]:
# Register the mesh to the original point cloud
scales = np.arange(0.8, 1.25, 0.025)
# scales = np.meshgrid(scales, scales, scales)
# scales = np.array(scales).reshape(3, -1).T
# print(f"Number of scales: {scales[..., None].shape}")
transformation = multiscale_registration(mesh_pcd, partial_pcd, scales, voxel_size=0.0025)

mesh_pcd.transform(transformation)

# Compute the chamfer distance
cd_o3d = (np.mean(complete_pcd.compute_point_cloud_distance(mesh_pcd)) + \
    np.mean(mesh_pcd.compute_point_cloud_distance(complete_pcd))) / 2
print(f"Chamfer Distance: {cd_o3d}")

# from utils.chamfer_python import distChamfer
# cd_left, cd_right, index_left, index_right = distChamfer(torch.tensor(np.asarray(complete_pcd.points)).unsqueeze(0), 
#                  torch.tensor(np.asarray(mesh_pcd.points)).unsqueeze(0))
# print(cd_left.shape, cd_right.shape)
# cd = (cd_left.sqrt().mean(dim=1) + cd_right.sqrt().mean(dim=1))
# print(f"Chamfer Distance (PyTorch): {cd.item()}")

[Open3D WARNING] Too few correspondences (479) after mutual filter, fall back to original correspondences.
[Open3D WARNING] Too few correspondences (479) after mutual filter, fall back to original correspondences.
[Open3D WARNING] Too few correspondences (479) after mutual filter, fall back to original correspondences.
[Open3D WARNING] Too few correspondences (479) after mutual filter, fall back to original correspondences.
[Open3D WARNING] Too few correspondences (479) after mutual filter, fall back to original correspondences.
[Open3D WARNING] Too few correspondences (478) after mutual filter, fall back to original correspondences.
[Open3D WARNING] Too few correspondences (480) after mutual filter, fall back to original correspondences.
[Open3D WARNING] Too few correspondences (481) after mutual filter, fall back to original correspondences.
[Open3D WARNING] Too few correspondences (478) after mutual filter, fall back to original correspondences.
[Open3D WARNING] Too few corresponden

In [19]:

draw([mesh_pcd, partial_pcd, complete_pcd])

[Open3D INFO] Window window_6 created.


WebVisualizer(window_uid='window_6')